In [38]:
# adding project dir to python path

import sys
import os

path = os.path.dirname(os.getcwd())
path, module = path[:path.rindex("\\")], path[path.rindex("\\")+1:]

print(path, module)
if path not in sys.path:
    sys.path.insert(0, path)
    print("inserted")
else:
    print("already exists")


C:\Users\Danish Amin\PycharmProjects TF_NeRF
already exists


In [34]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output, display
from time import sleep
from TF_NeRF.trainingV3 import get_rays, render#, LEVEL1, LEVEL2
from TF_NeRF.load import *
from TF_NeRF.times import Timings


tf.enable_eager_execution()
print("All import success")

All import success


In [29]:
# load model
model = load_model("model_L10_L0_H_S64_29p23.h5")

2022-06-20 19:29:40,199 - model loaded successfully from C:\Users\Danish Amin\PycharmProjects\TF_NeRF\models\model_L10_L0_H_S64_29p23.h5


In [30]:
# define any globals
from load import load_data
H, W = 100, 100
focal = tf.constant(load_data()['focal'], dtype=tf.float32)
samples = 64

2022-06-20 19:29:45,336 - data loaded successfully from C:\Users\Danish Amin\PycharmProjects\TF_NeRF\data\tiny_nerf_data.npz


In [31]:
# progress bar...

import time

def progress(x, total, string="", m=50.0):
    clear_output(wait=True)
    n = (m*x)/total
    complete = '='
    incomplete = '-'
    print(f"[{complete*int(n)}>{incomplete*int(m-int(n)-1)}] {1+int(100*(n)/m)}% completed {string}.")
    
for i in range(100):
    progress(i, 100)

[=================================================>] 100% completed .


In [32]:
%matplotlib inline
from ipywidgets import interactive, widgets
from IPython.display import clear_output, display
from time import sleep
from math import ceil


trans_t = lambda t : tf.convert_to_tensor([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,t],
    [0,0,0,1],
], dtype=tf.float32)

rot_phi = lambda phi : tf.convert_to_tensor([
    [1,0,0,0],
    [0,tf.cos(phi),-tf.sin(phi),0],
    [0,tf.sin(phi), tf.cos(phi),0],
    [0,0,0,1],
], dtype=tf.float32)

rot_theta = lambda th : tf.convert_to_tensor([
    [tf.cos(th),0,-tf.sin(th),0],
    [0,1,0,0],
    [tf.sin(th),0, tf.cos(th),0],
    [0,0,0,1],
], dtype=tf.float32)


def pose_spherical(theta, phi, radius):
    c2w = trans_t(radius)
    c2w = rot_phi(phi/180.*np.pi) @ c2w
    c2w = rot_theta(theta/180.*np.pi) @ c2w
    c2w = np.array([[-1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]]) @ c2w
    return c2w


def f(**kwargs):
    c2w = pose_spherical(**kwargs)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    rgb = tf.reshape(render(model, rays_o, rays_d, 2., 6., samples=samples, level1=10, level2=0), (H, W, 3))
    # rgb = render(model, rays_o, rays_d, near=2., far=6., samples=samples)
    img = np.clip(rgb,0,1)
    
    plt.figure(2, figsize=(20,6))
    plt.imshow(img)
    plt.show()
    

sldr = lambda v, mi, ma: widgets.FloatSlider(
    value=v,
    min=mi,
    max=ma,
    step=.01,
)

names = [
    ['theta', [100., 0., 360]],
    ['phi', [-30., -90, 0]],
    ['radius', [4., 3., 5.]],
]

interactive_plot = interactive(f, **{s[0] : sldr(*s[1]) for s in names})
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=100.0, description='theta', max=360.0, step=0.01), FloatSlider(value=-…

In [35]:
# video generator

from times import Timings
frames = np.zeros((120, H, W, 3))
frames = []

d = Timings()
for i, th in enumerate(np.linspace(0., 360., 120, endpoint=False)):
    c2w = pose_spherical(th, -30., 4.)
    rays_o, rays_d = get_rays(H, W, focal, c2w[:3,:4])
    d.get('render')
    rgb  = render(model, rays_o, rays_d, 2., 6., samples=samples, level1=10, level2=0)
    # rgb = render(model, rays_o, rays_d, near=2., far=6., samples=samples)
    d.get('render')
    d.get('save')
    frames.append((255*np.clip(rgb,0,1)).astype(np.uint8))
    d.get('save')
    progress(th, 360, string=
        f"render: {(d.d['render'][-1] - d.d['render'][-2])/1e9}, save: {(d.d['save'][-1] - d.d['save'][-2])/1e9}")


[=================================================>] 100% completed render: 2.5149127, save: 3.0593704.


In [41]:
import imageio
f = os.path.join(path, module, "videos", 'video_L10_L0_H_S64_29p23_testing.mp4')
imageio.mimwrite(f, frames, fps=30, quality=7)


2022-06-20 19:45:07,982 - IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (100, 100) to (112, 112) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [42]:
from IPython.display import HTML
from base64 import b64encode

f = os.path.join(path, module, "videos", 'video_L10_L0_H_S64_29p23_testing.mp4')

mp4 = open(f,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls autoplay loop>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)